<a href="https://colab.research.google.com/github/vatshivam/CU-course-similarity/blob/main/course_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install networkx
!pip install textwrap3
!pip install chart_studio
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from google.colab import files
import io
import plotly.figure_factory as ff
import plotly.express as px
import networkx as nx
import plotly.graph_objs as go
import plotly.io
import textwrap3
import chart_studio.plotly as py
import chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=5ae282f30fc3200faf83c698c65da14be98dec71491eddb445354d1a901cd1c7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd9

In [ ]:
uploaded = files.upload()

Saving Active Courses.xlsx to Active Courses.xlsx


In [ ]:
xlsx_file = io.BytesIO(uploaded.get('Active Courses.xlsx'))

In [ ]:
chart_studio.tools.set_credentials_file(username='shivam360d', api_key='DphUQ7ZSGpx5fcv21nBT')

In [ ]:
df = pd.read_excel(xlsx_file)

In [ ]:
df.head()

,EFFDT,EFF STATUS,SBJCT CD,CATALOG NUM,CRSE ID,CRSE LD,CRSE TTL LD,CRSE CATALOG LD,GRADE BASIS CD,SSR COMP CD,CATALOG PRINT,ACAD CAR CD,ACAD GRP CD,ACAD ORG CD,INSTRCTN MODE LD,MIN UNITS,MAX UNITS
0,2016-05-01,A,XFCR,2999SA,100001,Lower Division General Credit,Lower Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,0.1,99.0
1,2016-05-01,A,XFCR,2999TC,100001,Lower Division General Credit,Lower Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,0.1,99.0
2,2016-05-01,A,XFCR,3999SA,100002,Upper Division General Credit,Upper Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,3.0,3.0
3,2016-05-01,A,XFCR,3999TC,100002,Upper Division General Credit,Upper Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,3.0,3.0
4,2016-05-01,A,XFCR,4999SA,100003,Upper Division General Credit,Upper Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,0.1,99.0


In [ ]:
df.head().to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>EFFDT</th>\n      <th>EFF STATUS</th>\n      <th>SBJCT CD</th>\n      <th>CATALOG NUM</th>\n      <th>CRSE ID</th>\n      <th>CRSE LD</th>\n      <th>CRSE TTL LD</th>\n      <th>CRSE CATALOG LD</th>\n      <th>GRADE BASIS CD</th>\n      <th>SSR COMP CD</th>\n      <th>CATALOG PRINT</th>\n      <th>ACAD CAR CD</th>\n      <th>ACAD GRP CD</th>\n      <th>ACAD ORG CD</th>\n      <th>INSTRCTN MODE LD</th>\n      <th>MIN UNITS</th>\n      <th>MAX UNITS</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>2016-05-01</td>\n      <td>A</td>\n      <td>XFCR</td>\n      <td>2999SA</td>\n      <td>100001</td>\n      <td>Lower Division General Credit</td>\n      <td>Lower Division General Credit</td>\n      <td>-</td>\n      <td>OPT</td>\n      <td>LEC</td>\n      <td>N</td>\n      <td>UGRD</td>\n      <td>CRSS</td>\n      <td>B-CRSS</td>\n      <td>In Person</

In [ ]:
''' Removing rows with no course catalog information '''
df = df[df['CRSE CATALOG LD']!='-']

In [ ]:
''' Removing duplicate courses with the same course catalog '''
df.drop_duplicates(subset=['CRSE CATALOG LD'],inplace=True)

In [ ]:
'''Removing courses with same name and department,however differing in the level(especially 4000 vs 5000)'''
df.drop_duplicates(subset=['SBJCT CD','CRSE TTL LD'],inplace=True)

In [ ]:
df.shape

(7272, 17)

In [ ]:
df.drop_duplicates(subset=['CRSE TTL LD'],inplace=True)

In [ ]:
df.shape

(6503, 17)

In [ ]:
#patterns: subject name | subject name 1
# description has "Same as XXXX 1234"
# same name different department

In [ ]:
df.head()

,EFFDT,EFF STATUS,SBJCT CD,CATALOG NUM,CRSE ID,CRSE LD,CRSE TTL LD,CRSE CATALOG LD,GRADE BASIS CD,SSR COMP CD,CATALOG PRINT,ACAD CAR CD,ACAD GRP CD,ACAD ORG CD,INSTRCTN MODE LD,MIN UNITS,MAX UNITS
12,2018-08-02,A,ACCT,3220,100190,Corporate Financial Rprtng 1,Corporate Financial Reporting 1,First of a two-course sequence intended to pro...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
13,2017-08-01,A,ACCT,3230,100191,Corporate Financial Rprtng 2,Corporate Financial Reporting 2,Second in a two-course sequence building and e...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
14,2018-08-02,A,ACCT,3320,100192,Cost Management,Cost Management,Provides cost analysis for the support of mana...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
17,2017-08-01,A,ACCT,4240,100194,Advanced Financial Accounting,Advanced Financial Accounting,Examines advanced financial accounting theory ...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
18,2019-08-01,A,ACCT,4250,100195,Financial Statement Analysis,Financial Statement Analysis,Focuses on the use of U.S. and international a...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0


In [ ]:
def similarity(model,course1,course2,course_id1,course_id2,embedding_dic):

  '''Function to compute embedding of two courses and return their similarity. It also stores the embeddings for future use'''

  # calculating and storing embeddings
  if course_id1 not in embedding_dic.keys():
    embedding1 = model.encode(course1, convert_to_tensor=True)
    embedding_dic[course_id1]=embedding1.numpy()

  if course_id2 not in embedding_dic.keys():
    embedding2 = model.encode(course2, convert_to_tensor=True)
    embedding_dic[course_id2]=embedding2.numpy()

  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding_dic[course_id1], embedding_dic[course_id2])

  return cosine_scores,embedding_dic

In [ ]:
def courses_similarity(dept1,dept2,model='stsb-roberta-large',n=10):

  '''Fuction returns top n similar courses for two different departments'''

  embedding_dic={}
  scores=[]

  # initialize the embedding model
  model = SentenceTransformer('stsb-roberta-large')

  # pairwise combinations of courses
  for index,r1 in df[df['SBJCT CD']==dept1].iterrows():
    for index,r2 in df[df['SBJCT CD']==dept2].iterrows():

      course1_id = r1['SBJCT CD']+r1['CATALOG NUM']
      course2_id = r2['SBJCT CD']+r2['CATALOG NUM']

      course1 = r1['CRSE CATALOG LD']
      course2 = r2['CRSE CATALOG LD']

      # computing similarity
      score, embedding_dic = similarity(model,course1,course2,course1_id,course2_id,embedding_dic)
      score = round(score.item(),2)

      key = course1_id+' '+course2_id
      print("Similarity scores between:",key,score)
      scores.append((key,score))

  # sorting the scores and extracting top n entries
  scores = sorted(scores,key= lambda x: x[1],reverse=True)

  return scores,embedding_dic

In [ ]:
res, embedding_dic = courses_similarity('DTSC','MSBX')

In [ ]:
# creating CSV file and storing the scores
name = res[0][0].split()
name = name[0][:4]+'_'+name[1][:4]+'.csv'
df = pd.DataFrame(res,columns=['Courses',"Score"])
df[["Course 1","Course 2"]]=df.Courses.str.split(expand=True)
df.drop(columns=['Courses'],inplace=True)
df.to_csv(name,encoding = 'utf-8-sig')
files.download(name)

In [ ]:
def create_dendrogram(embedding_dic):

  '''Function to create dendograms leveraging embeddings dictionary'''

  # getting labels
  courses = list(embedding_dic.keys())
  embeddings_array = np.empty((0,1024))

  # preparing embeddings
  for course in courses:
    embeddings_array = np.append(embeddings_array,embedding_dic[course])
  embeddings_array = embeddings_array.reshape(len(embedding_dic),1024)

  # creating dendogram
  fig = ff.create_dendrogram(embeddings_array,labels=courses)
  fig.update_layout(title={'text':'Similarity heirarchy','x':0.5,'y':0.95},xaxis_title="Courses",yaxis_title="Count",width=800, height=500)
  fig.show()

In [ ]:
create_dendrogram(embedding_dic)

In [ ]:
res.sort(key=lambda a: a[0].split()[0])

In [ ]:
score_np = np.array([score[1] for score in res])

In [ ]:
score_np = score_np.reshape(df[df['SBJCT CD']=='MSBX'].shape[0],df[df['SBJCT CD']=='DTSC'].shape[0])

In [ ]:
import plotly.express as px
y_labels = [x[0].split()[1] for x in res[:score_np.shape[0]]]
x_labels = list(set([x[0].split()[0] for x in res]))
fig = px.imshow(score_np,y=y_labels,x=x_labels,labels = dict(color = "Similarity"),color_continuous_scale=["white", "black"])
fig.update_layout(title={'text':'Similarity heatmap','x':0.5,'y':0.95},xaxis_title="Courses",yaxis_title="Courses",)
fig.show()

Analyzing courses having specific keywords in their course description

In [ ]:
def get_subset(data,word):
  data['course_id'] = data['SBJCT CD']+data['CATALOG NUM']
  return data[(data['CRSE CATALOG LD'].str.lower().str.contains(word))][["SBJCT CD","CATALOG NUM","CRSE CATALOG LD","course_id",'CRSE TTL LD']]

# df['course_id'] = df['SBJCT CD']+df['CATALOG NUM']
# calculus_data = df[(df['CRSE CATALOG LD'].str.lower().str.contains('calculus'))][["SBJCT CD","CATALOG NUM","CRSE CATALOG LD","course_id",'CRSE TTL LD']]
# ml_data = df[(df['CRSE CATALOG LD'].str.lower().str.contains('machine learning'))][["SBJCT CD","CATALOG NUM","CRSE CATALOG LD","course_id",'CRSE TTL LD']]
# stats_data = df[(df['CRSE CATALOG LD'].str.lower().str.contains('statistics'))][["SBJCT CD","CATALOG NUM","CRSE CATALOG LD","course_id",'CRSE TTL LD']]

In [ ]:
def similarity_all_pairs(data):
  embedding_dic={}
  scores = []
  model = SentenceTransformer('stsb-roberta-large')
  for i in range(data.shape[0]):
    for j in range(data.shape[0]):
      course1 = data.iloc[i]['CRSE CATALOG LD']
      course2 = data.iloc[j]['CRSE CATALOG LD']
      course_id1 = data.iloc[i]['course_id']
      course_id2 = data.iloc[j]['course_id']

      # computing similarity
      score, embedding_dic = similarity(model,course1,course2,course_id1,course_id2,embedding_dic)
      score = round(score.item(),2)
      key = course_id1+' '+course_id2

      print("Similarity scores between:",key,score)
      scores.append((key,score))
  return scores,embedding_dic

In [ ]:
def get_labels(list_of_tuples):
  labels = []
  for score in list_of_tuples:
    labels.append(score[0].split()[1])
  return labels

In [ ]:
def score_heatmap(scores,total_courses):
  score_np = np.array([score[1] for score in scores])
  score_np = score_np.reshape(total_courses,total_courses)
  labels = get_labels(scores[:total_courses])
  fig = px.imshow(score_np,x=labels,y=labels,color_continuous_scale=['white','black'],labels=dict(color='Score'))
  fig.update_layout(title={'text':'Similarity heatmap','x':0.5,'y':0.95},xaxis_title="Courses",yaxis_title="Courses",xaxis=dict(showticklabels=False),yaxis=dict(showticklabels=False))
  fig.show()
  return score_np,labels

In [ ]:
def course_description_wrapped(labels,data):
  course_desc = []
  course_name = []
  for label in labels:
    dept = label[:4]
    code = label[4:]
    course_name.append(data[(data['CATALOG NUM']==code) & (data['SBJCT CD']==dept)]['CRSE TTL LD'].values[0])
    course_description = data[(data['CATALOG NUM']==code) & (data['SBJCT CD']==dept)]['CRSE CATALOG LD'].values[0]
    course_desc.append("<br>".join(textwrap3.wrap(course_description,50)))
  return course_name, course_desc

In [ ]:
data = get_subset(df,'statistics')
scores,embeddings = similarity_all_pairs(data)

Streaming output truncated to the last 5000 lines.
Similarity scores between: IPHY2800 IPHY4540 0.55
Similarity scores between: IPHY2800 IPHY6680 0.56
Similarity scores between: IPHY2800 LAWS6803 0.55
Similarity scores between: IPHY2800 MATH1120 0.38
Similarity scores between: IPHY2800 MATH1310 0.71
Similarity scores between: IPHY2800 MBAC6010 0.32
Similarity scores between: IPHY2800 MBAC6011 0.32
Similarity scores between: IPHY2800 MCEN3047 0.39
Similarity scores between: IPHY2800 MUSC7113 0.41
Similarity scores between: IPHY2800 PSCI7085 0.56
Similarity scores between: IPHY2800 PSYC2111 0.46
Similarity scores between: IPHY2800 PSYC5741 0.39
Similarity scores between: IPHY2800 SLHS7000 0.5
Similarity scores between: IPHY2800 SOCY5111 0.46
Similarity scores between: IPHY2800 SOCY7111 0.4
Similarity scores between: IPHY2800 PR881088U 0.37
Similarity scores between: IPHY2800 GEOG4271 0.38
Similarity scores between: IPHY2800 IPHY4580 0.5
Similarity scores between: IPHY2800 ENVS2000 0.42
S

In [ ]:
score_matrix,labels = score_heatmap(scores,len(embeddings))
course_name,course_desc = course_description_wrapped(labels,data)

In [ ]:
# G = nx.to_networkx_graph(score_matrix)

In [ ]:
def rgb_to_hex(l):
    return '#{:02x}{:02x}{:02x}'.format(l[0], l[1], l[2])

In [ ]:
def create_node_trace(G):
    # collect node information from G to plot
    node_x = []
    node_y = []
    node_text = []
    node_color = []
    node_desc = []
    node_name = []

    for i, node in enumerate(G.nodes(data=True)):
        # get node x,y position and store
        x, y = node[1]['pos']
        node_x.append(x)
        node_y.append(y)

        node_text.append(node[1]['text'])
        node_color.append(node[1]['color'])
        node_desc.append(node[1]['course_desc'])
        node_name.append(node[1]['name'])

    # create node trace (i.e., scatter plot)
    # make it invisible by default
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        marker={'symbol': 'circle', 'size': 41,'color':node_color,'opacity':0.7},
        mode='markers+text',
        hoverinfo='text',
        hovertemplate='<b>%{customdata[0]}</b>:<br>%{customdata[1]}'+'<extra></extra>',
        text=node_text,
        textfont={'size':9,'color':'black'},
        customdata=np.stack((node_name,node_desc),axis=-1),
        hoverlabel_align = 'auto',
        visible=False
    )

    return node_trace

In [ ]:
def create_edge_trace(G):
    # collect edges information from G to plot
    edge_weight = []
    edge_text = []
    edge_pos = []
    edge_color = []

    for edge in G.edges(data=True):

        # edge is line connecting two points
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_pos.append([[x0, x1, None], [y0, y1, None]])

        # edge line color when drawn
        edge_color.append("grey")

    # there is a trace for each edge
    edge_traces = []
    for i in range(len(edge_pos)):

        # edge line width
        line_width = 1

        # is scatter because it is line connecting two points
        trace = go.Scatter(
            x=edge_pos[i][0], y=edge_pos[i][1],
            line=dict(width=line_width, color=edge_color[i]),
            mode='lines',
            visible=False,
            opacity=0.5
        )
        edge_traces.append(trace)

    return edge_traces


In [ ]:
def filter_similarity_matrix_at_step(square_matrix, step_value):
    # copy matrix
    aux = square_matrix.copy()

    # set as NaN all values equal to or below threshold value
    aux[aux <= step_value] = np.nan

    # return filtered matrix
    return aux

In [ ]:
def get_interactive_slider_similarity_graph(square_matrix, slider_values, color_mapping, node_text=None, yaxisrange=None, xaxisrange=None,node_desc=None,node_name=None):

    # Create figure with plotly
    fig = go.Figure()

    # key: slider value
    # value: list of traces to display for that slider value
    slider_dict = {}

    # total number of traces
    total_n_traces = 0

    # node positions on plot
    #node_pos = None

    # for each possible value in the slider, create and store traces (i.e., plots)
    for i, step_value in enumerate(slider_values):
        # update similarity matrix for the current step
        aux = filter_similarity_matrix_at_step(square_matrix, step_value)

        # create nx graph from sim matrix
        G = nx.to_networkx_graph(aux)

        # remove edges for 0 weight (NaN)
        G.remove_edges_from([(a, b) for a, b, attrs in G.edges(data=True) if np.isnan(attrs["weight"])])

        # remove nodes having no connections
        nodes_to_remove = []
        for n in range(len(labels)):
            neighbors = list(G.neighbors(n))
            if len(neighbors)==1:
                nodes_to_remove.append(n)

        G.remove_nodes_from(nodes_to_remove)

        # assign node positions if None
        # node_pos = nx.nx_pydot.graphviz_layout(G)
        # node_pos = nx.spring_layout(G,k=1,seed=10)
        # node_pos = nx.circular_layout(G,scale=1)
        # node_pos = nx.kamada_kawai_layout(G)
        # node_pos = nx.random_layout(G,seed=10)
        # node_pos = nx.shell_layout(G)
        # node_pos = nx.spiral_layout(G)
        # node_pos = nx.bipartite_layout(G,nodes=G.nodes)
        node_pos = nx.nx_pydot.pydot_layout(G, prog='neato')
        # node_pos = nx.planar_layout(G.nodes)

        # populate nodes with meta information
        for node in G.nodes(data=True):

            # node position
            node[1]['pos'] = node_pos[node[0]]

            # node text on hover if any is specified else is empty
            if (node_text is not None) and (node_desc is not None):
                node[1]['text'] = node_text[node[0]]
                node[1]['course_desc'] = node_desc[node[0]]
                node[1]['name'] = node_name[node[0]]
            else:
                node[1]['text'] = ""
                node[1]['course_desc'] = ""

            # node color
            node[1]['color'] = color_mapping[node[1]['text'][:4]]


        # create edge taces (each edge is a trace, thus this is a list)
        edge_traces = create_edge_trace(G)

        # create node trace (a single trace for all nodes, thus it is not a list)
        node_trace = create_node_trace(G)

        # store edge+node traces as single list for the current step value
        slider_dict[step_value] = edge_traces + [node_trace]

        # keep count of the total number of traces
        total_n_traces += len(slider_dict[step_value])

        # make sure that the first slider value is active for visualization
        if i == 0:
            for trace in slider_dict[step_value]:
            # make visible
                trace.visible = True


    # Create steps objects (one step per step_value)
    steps = []
    for step_value in slider_values:

        # count traces before adding new traces
        n_traces_before_adding_new = len(fig.data)

        # add new traces
        fig.add_traces(slider_dict[step_value])

        step = dict(
            # update figure when this step is active
            method="update",
            # make all traces invisible
            args=[{"visible": [False] * total_n_traces}],
            # label on the slider
            label=str(round(step_value, 3)),
        )

        # only toggle this step's traces visible, others remain invisible
        n_traces_for_step_value = len(slider_dict[step_value])
        for i in range(n_traces_before_adding_new, n_traces_before_adding_new + n_traces_for_step_value):
            step["args"][0]["visible"][i] = True

        # store step object in list of many steps
        steps.append(step)

    # create slider with list of step objects
    slider = [dict(
        active=0,
        steps=steps
    )]

    # add slider to figure and create layout
    fig.update_layout(
        sliders=slider,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(range=xaxisrange, showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(range=yaxisrange, showgrid=False, zeroline=False, showticklabels=False),
        width=750, height=750,
    )

    return fig


In [ ]:
unique_dept = np.unique(df['SBJCT CD'])
color_mapping = {}
for i in range(len(unique_dept)):
  color_value = list(np.random.choice(range(255),size=3))
  color_value = rgb_to_hex(color_value)
  color_mapping[unique_dept[i]]=color_value

In [ ]:
# define slider steps (i.e., threshold values)
slider_steps = np.arange(0.65, 1, 0.05)

# get the slider figure
fig = get_interactive_slider_similarity_graph(
    score_matrix,
    slider_steps,
    color_mapping,
    node_text = labels,
    node_desc = course_desc,
    node_name = course_name
)

# plot it
fig.show()

<ipython-input-32-d527320f6cf9>:45: DeprecationWarning:

nx.nx_pydot.pydot_layout depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723

<ipython-input-32-d527320f6cf9>:45: DeprecationWarning:

nx.nx_pydot.pydot_layout depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723

<ipython-input-32-d527320f6cf9>:45: DeprecationWarning:

nx.nx_pydot.pydot_layout depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723

<ipython-input-32-d527320f6cf9>:45: DeprecationWarning:

nx.nx_pydot.pydot_layout depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723

<ipython-input-32-d527320f6cf9>:45: DeprecationWarning:

nx.nx_pydot.pydot_layout depends on the pydot package, whic

In [ ]:
# html_calculus = plotly.io.to_html(fig,full_html=False)
# text_file = open("calculus_plot.txt", "w")
# n = text_file.write(html_calculus)
# text_file.close()
# files.download('calculus_plot.txt')

In [ ]:
py.plot(fig, filename = 'stats_data', auto_open=False)

'https://plotly.com/~shivam360d/13/'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>